In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from bioinfokit.analys import stat
import researchpy as rp
import numpy as np
import datetime
import fiona
from shapely.geometry import shape, Point, Polygon, MultiPolygon, LineString
import shapefile
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import compute_sample_weight

In [2]:
# file_path = "./processed.csv"
# data = pd.read_csv(file_path)

In [3]:
file_path = "./processed_all_borough.csv"
borough_df = pd.read_csv(file_path)

/home/milica/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
file_path = "./processed_street.csv"
street_df = pd.read_csv(file_path)

In [5]:
print(borough_df.shape)
borough_df.head()

(1026365, 17)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",WEST STREET,ALBANY STREET,NaN,Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",CENTRAL AVENUE,NaN,NaN,Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",ATLANTIC AVENUE,HICKS STREET,NaN,Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",43 STREET,17 AVENUE,NaN,Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",MAJOR DEEGAN EXPRESSWAY,NaN,NaN,Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0


In [6]:
sum_nan = borough_df.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

                                 #NaN
ZIP CODE                       278136
ON STREET NAME                 224405
CROSS STREET NAME              421155
OFF STREET NAME                828433
CONTRIBUTING FACTOR VEHICLE 1    3327
CONTRIBUTING FACTOR VEHICLE 2  147482
VEHICLE TYPE CODE 1              6219
VEHICLE TYPE CODE 2            181654


In [7]:
print(street_df.shape)
street_df.head()

(1026643, 29)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",WEST STREET,ALBANY STREET,NaN,...,Passing Too Closely,NaN,NaN,NaN,4247215,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",CENTRAL AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",ATLANTIC AVENUE,HICKS STREET,NaN,...,Unspecified,NaN,NaN,NaN,4246353,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",43 STREET,17 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4215020,Sedan,Sedan,NaN,NaN,NaN
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",MAJOR DEEGAN EXPRESSWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4246365,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [8]:
sum_nan = street_df.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

                                  #NaN
ZIP CODE                        278159
ON STREET NAME                       4
CROSS STREET NAME               421182
OFF STREET NAME                 828691
CONTRIBUTING FACTOR VEHICLE 1     3327
CONTRIBUTING FACTOR VEHICLE 2   147519
CONTRIBUTING FACTOR VEHICLE 3   961785
CONTRIBUTING FACTOR VEHICLE 4  1013037
CONTRIBUTING FACTOR VEHICLE 5  1023061
VEHICLE TYPE CODE 1               6220
VEHICLE TYPE CODE 2             181696
VEHICLE TYPE CODE 3             964115
VEHICLE TYPE CODE 4            1013554
VEHICLE TYPE CODE 5            1023188


In [9]:
street_df.COLLISION_ID = street_df.COLLISION_ID.astype(int)
street_df.COLLISION_ID = street_df.COLLISION_ID.astype(object)

In [10]:
borough_df.COLLISION_ID = borough_df.COLLISION_ID.astype(int)
borough_df.COLLISION_ID = borough_df.COLLISION_ID.astype(object)

In [11]:
borough_df = borough_df.loc[borough_df.COLLISION_ID.isin(street_df.COLLISION_ID)]
street_df = street_df.loc[street_df.COLLISION_ID.isin(borough_df.COLLISION_ID)]

In [12]:
street_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",WEST STREET,ALBANY STREET,NaN,...,Passing Too Closely,NaN,NaN,NaN,4247215,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",CENTRAL AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",ATLANTIC AVENUE,HICKS STREET,NaN,...,Unspecified,NaN,NaN,NaN,4246353,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",43 STREET,17 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4215020,Sedan,Sedan,NaN,NaN,NaN
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",MAJOR DEEGAN EXPRESSWAY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4246365,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [13]:
borough_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",WEST STREET,ALBANY STREET,NaN,Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",CENTRAL AVENUE,NaN,NaN,Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",ATLANTIC AVENUE,HICKS STREET,NaN,Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",43 STREET,17 AVENUE,NaN,Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",MAJOR DEEGAN EXPRESSWAY,NaN,NaN,Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0


In [14]:
print(borough_df.shape)
print(street_df.shape)

(1026365, 17)
(1026365, 29)


In [15]:
new_df = street_df[['COLLISION_ID', 'ON STREET NAME']]
new_df.head()

,COLLISION_ID,ON STREET NAME
0,4247215,WEST STREET
1,4246795,CENTRAL AVENUE
2,4246353,ATLANTIC AVENUE
3,4215020,43 STREET
4,4246365,MAJOR DEEGAN EXPRESSWAY


In [16]:
borough_df = borough_df.drop(columns=['ON STREET NAME'])
borough_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",ALBANY STREET,NaN,Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",NaN,NaN,Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",HICKS STREET,NaN,Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",17 AVENUE,NaN,Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",NaN,NaN,Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0


In [17]:
merged = pd.merge(borough_df, new_df, on=['COLLISION_ID'])

In [29]:
index_names = merged[(pd.isna(merged["ON STREET NAME"]))].index
merged = merged.drop(index_names) 

In [30]:
merged.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED,ON STREET NAME,HOUR
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",ALBANY STREET,NaN,Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0,WEST STREET,20
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",NaN,NaN,Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0,CENTRAL AVENUE,16
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",HICKS STREET,NaN,Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0,ATLANTIC AVENUE,12
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",17 AVENUE,NaN,Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0,43 STREET,13
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",NaN,NaN,Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0,MAJOR DEEGAN EXPRESSWAY,11


In [32]:
sum_nan = merged.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

                                 #NaN
ZIP CODE                       278135
CROSS STREET NAME              421152
OFF STREET NAME                828433
CONTRIBUTING FACTOR VEHICLE 1    3327
CONTRIBUTING FACTOR VEHICLE 2  147481
VEHICLE TYPE CODE 1              6219
VEHICLE TYPE CODE 2            181653


In [33]:
data = merged
data.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CROSS STREET NAME,OFF STREET NAME,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED,ON STREET NAME,HOUR
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",ALBANY STREET,NaN,Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0,WEST STREET,20
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",NaN,NaN,Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0,CENTRAL AVENUE,16
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",HICKS STREET,NaN,Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0,ATLANTIC AVENUE,12
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",17 AVENUE,NaN,Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0,43 STREET,13
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",NaN,NaN,Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0,MAJOR DEEGAN EXPRESSWAY,11


In [34]:
# data['CRASH DATE'] = pd.to_datetime(data['CRASH DATE'])
data['HOUR'] = data['CRASH TIME'].apply(lambda x: x.split(':')[0])
data['HOUR'] = data['HOUR'].astype(int)

In [35]:
data = data.drop(columns=['CROSS STREET NAME', 'OFF STREET NAME'])
data.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED,ON STREET NAME,HOUR
0,2019-11-24,20:00,MANHATTAN,10006.0,40.70983,-74.01468,"(40.70983, -74.01468)",Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0,WEST STREET,20
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",Failure to Yield Right-of-Way,Unspecified,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,0,0,CENTRAL AVENUE,16
2,2019-11-24,12:20,BROOKLYN,11201.0,40.69119,-73.99780,"(40.69119, -73.9978)",Unsafe Lane Changing,Unspecified,4246353,Sedan,Station Wagon/Sport Utility Vehicle,2,0,ATLANTIC AVENUE,12
3,2019-09-26,13:07,BROOKLYN,11204.0,40.63291,-73.97957,"(40.63291, -73.97957)",Backing Unsafely,Unspecified,4215020,Sedan,Sedan,0,0,43 STREET,13
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",Unspecified,Unspecified,4246365,Station Wagon/Sport Utility Vehicle,Sedan,0,0,MAJOR DEEGAN EXPRESSWAY,11


In [36]:
file_weather = "./datasets/weather.csv"
weather = pd.read_csv(file_weather)

In [37]:
weather['CRASH DATE'] = weather['Datetime'].apply(lambda x: x.split(' ')[0])
weather['HOUR'] = weather['Datetime'].apply(lambda x: x.split(' ')[1].split(':')[0])
weather['HOUR'] = weather['HOUR'].astype(int)

In [38]:
# weather['CRASH DATE'] = pd.to_datetime(weather['CRASH DATE'])
weather.head()

,Datetime,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,WindGust,Pressure,Precip.,Condition,Day/Night,Clouds,CRASH DATE,HOUR
0,2014-01-01 00:00,27,16.0,6.0,41,WNW,14.0,24.0,30.26,NaN,Fair,N,CLR,2014-01-01,0
1,2014-01-01 01:00,26,13.0,4.0,39,W,17.0,28.0,30.27,NaN,Fair,N,CLR,2014-01-01,1
2,2014-01-01 02:00,25,13.0,6.0,45,WNW,15.0,22.0,30.29,NaN,Fair,N,CLR,2014-01-01,2
3,2014-01-01 03:00,25,13.0,6.0,45,W,15.0,21.0,30.32,NaN,Fair,N,CLR,2014-01-01,3
4,2014-01-01 04:00,25,14.0,6.0,45,W,13.0,NaN,30.33,NaN,Fair,N,CLR,2014-01-01,4


In [39]:
sum_nan = weather.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

            #NaN
FeelsLike     18
Dew           45
Wind          28
WindSpeed    114
WindGust   41708
Pressure      23
Precip.     1423
Clouds         2


In [40]:
weather = weather.drop(columns='WindGust')

In [41]:
data.shape

(1026362, 16)

In [42]:
merged = pd.merge(data, weather, on=['CRASH DATE', 'HOUR'])

In [43]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
merged.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED,ON STREET NAME,HOUR,Datetime,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,Pressure,Precip.,Condition,Day/Night,Clouds
0,2019-11-24,20:00,MANHATTAN,10006.0,40.709830,-74.014680,"(40.70983, -74.01468)",Passing Too Closely,Passing Too Closely,4247215,Sedan,Station Wagon/Sport Utility Vehicle,0,0,WEST STREET,20,2019-11-24 20:00,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
1,2019-11-24,20:40,QUEENS,11354.0,40.758960,-73.829850,"(40.75896, -73.82985)",Failure to Yield Right-of-Way,NaN,4246722,Station Wagon/Sport Utility Vehicle,NaN,2,0,MAIN STREET,20,2019-11-24 20:00,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
2,2019-11-24,20:33,BROOKLYN,11235.0,40.588440,-73.949326,"(40.58844, -73.949326)",Driver Inattention/Distraction,Unspecified,4246879,Sedan,Station Wagon/Sport Utility Vehicle,0,0,OCEAN AVENUE,20,2019-11-24 20:00,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
3,2019-11-24,20:30,BROOKLYN,11210.0,40.624798,-73.942650,"(40.624798, -73.94265)",Following Too Closely,Unspecified,4247059,Sedan,Sedan,0,0,AVENUE K,20,2019-11-24 20:00,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
4,2019-11-24,20:15,QUEENS,11414.0,40.663322,-73.854540,"(40.663322, -73.85454)",Driver Inattention/Distraction,Unspecified,4247534,Sedan,NaN,0,0,79 STREET,20,2019-11-24 20:00,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW


In [44]:
sum_nan = merged.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

                                 #NaN
ZIP CODE                       277907
CONTRIBUTING FACTOR VEHICLE 1    3323
CONTRIBUTING FACTOR VEHICLE 2  147337
VEHICLE TYPE CODE 1              6217
VEHICLE TYPE CODE 2            181483
FeelsLike                         176
Wind                              749
WindSpeed                         749
Pressure                          309
Clouds                              2


In [45]:
merged.shape

(1025439, 28)

### Izbaciti ili staviti srednju vrijednost

In [46]:
diff = set(data.COLLISION_ID) - set(merged.COLLISION_ID)

In [47]:
len(diff)

923

In [48]:
lost = data.loc[data.COLLISION_ID.isin(diff)]
lost.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,INJURED,KILLED,ON STREET NAME,HOUR
2909,2019-10-21,14:15,QUEENS,11372.0,40.754776,-73.892815,"(40.754776, -73.892815)",Driver Inattention/Distraction,NaN,4227096,Station Wagon/Sport Utility Vehicle,NaN,2,0,NORTHERN BOULEVARD,14
3709,2019-10-21,14:00,BRONX,NaN,40.825424,-73.923485,"(40.825424, -73.923485)",Passing Too Closely,Other Vehicular,4227013,Sedan,NaN,0,0,GRAND CONCOURSE,14
4629,2019-10-21,14:00,BRONX,NaN,40.847004,-73.832290,"(40.847004, -73.83229)",Driver Inattention/Distraction,Aggressive Driving/Road Rage,4230487,Sedan,E-Bik,2,0,CROSBY AVENUE,14
4995,2019-10-21,14:40,BRONX,10452.0,40.839832,-73.923920,"(40.839832, -73.92392)",Passing Too Closely,Unspecified,4227354,Bus,Station Wagon/Sport Utility Vehicle,0,0,WEST 169 STREET,14
5020,2019-10-21,14:00,BROOKLYN,11212.0,40.663147,-73.918840,"(40.663147, -73.91884)",Unspecified,NaN,4227097,Sedan,NaN,0,0,HOWARD AVENUE,14


In [49]:
len(lost.loc[lost['CRASH DATE'] == '2014-10-06'])

44

In [50]:
lost['CRASH DATE'].unique()

array(['2019-10-21', '2019-05-10', '2019-11-20', '2019-04-24',
       '2018-08-25', '2018-08-11', '2019-01-29', '2018-04-10',
       '2018-04-09', '2018-10-11', '2017-10-03', '2018-05-10',
       '2018-05-12', '2018-05-09', '2017-11-22', '2017-07-18',
       '2017-06-18', '2017-07-08', '2014-10-06', '2017-05-12',
       '2017-03-22', '2014-09-30', '2014-09-27', '2016-08-03',
       '2016-06-28', '2016-01-21', '2015-10-02', '2015-10-03',
       '2015-07-01', '2015-06-17', '2015-06-20', '2015-06-21'],
      dtype=object)

In [52]:
merged.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'INJURED', 'KILLED', 'ON STREET NAME', 'HOUR',
       'Datetime', 'Temperature', 'FeelsLike', 'Dew', 'Humidity', 'Wind',
       'WindSpeed', 'Pressure', 'Precip.', 'Condition', 'Day/Night', 'Clouds'],
      dtype='object')

In [53]:
model_data = merged.drop(columns=['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'Datetime', 'LOCATION'])

In [54]:
model_data.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,INJURED,KILLED,ON STREET NAME,HOUR,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,Pressure,Precip.,Condition,Day/Night,Clouds
0,2019-11-24,20:00,MANHATTAN,10006.0,40.709830,-74.014680,0,0,WEST STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
1,2019-11-24,20:40,QUEENS,11354.0,40.758960,-73.829850,2,0,MAIN STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
2,2019-11-24,20:33,BROOKLYN,11235.0,40.588440,-73.949326,0,0,OCEAN AVENUE,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
3,2019-11-24,20:30,BROOKLYN,11210.0,40.624798,-73.942650,0,0,AVENUE K,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW
4,2019-11-24,20:15,QUEENS,11414.0,40.663322,-73.854540,0,0,79 STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW


In [56]:
sum_nan = model_data.isnull().sum().to_frame()
sum_nan.columns=['#NaN']
sum_nan_thresh = sum_nan[sum_nan['#NaN'] > 0]
pd.set_option('display.max_rows', None)
print(sum_nan_thresh)

             #NaN
ZIP CODE   277907
FeelsLike     176
Wind          749
WindSpeed     749
Pressure      309
Clouds          2


### Create Severity column (0 - , 1 - injured > 0, 0 killed, 2 - killed > 0)

In [57]:
sev = pd.DataFrame()
sev['INJURED'] = (model_data['INJURED'] > 0).astype(int) * 1
sev['KILLED'] = (model_data['KILLED'] > 0).astype(int) * 2
model_data['SEVERITY'] = sev.max(axis=1)

In [58]:
model_data = model_data.drop(columns=['KILLED', 'INJURED'])
model_data.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,HOUR,Temperature,FeelsLike,Dew,Humidity,Wind,WindSpeed,Pressure,Precip.,Condition,Day/Night,Clouds,SEVERITY
0,2019-11-24,20:00,MANHATTAN,10006.0,40.709830,-74.014680,WEST STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
1,2019-11-24,20:40,QUEENS,11354.0,40.758960,-73.829850,MAIN STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,1
2,2019-11-24,20:33,BROOKLYN,11235.0,40.588440,-73.949326,OCEAN AVENUE,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
3,2019-11-24,20:30,BROOKLYN,11210.0,40.624798,-73.942650,AVENUE K,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0
4,2019-11-24,20:15,QUEENS,11414.0,40.663322,-73.854540,79 STREET,20,43,35.0,31.0,62,WNW,17.0,29.5,0.0,Fair,N,FEW,0


In [59]:
model_data.to_csv("./merged_crash_weather.csv", index=False)